In [1]:
import pandas as pd
import numpy as np
import datetime
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import requests
from bs4 import BeautifulSoup
import time
import re
from urllib.request import urlopen
import optuna.integration.lightgbm as lgb_o
from itertools import combinations, permutations
import matplotlib.pyplot as plt
from selenium import webdriver
from urllib.parse import urljoin
from chardet import detect
from datetime import datetime

In [2]:
%pip install chardet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 1.2 MB/s eta 0:00:0000:0100:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
class GetChokyo:
    def __init__(self):
        self.user_id = 'ken.osechi@gmail.com'
        self.password = 'keson0213'
        self.login_info = {
            "login_id": self.user_id,
            "pswd": self.password,
        }
        self.session = requests.session()
        self.url_login = "https://regist.netkeiba.com/account/?pid=login&action=auth"
        self.ses = self.session.post(self.url_login, data=self.login_info)
        self.cookie = self.ses.cookies

    def chokyo_one_horse(self, horse_id):
        url = 'https://db.sp.netkeiba.com/horse/training/' + horse_id

        html = self.session.get(url, cookies=self.cookie)
        html.encoding = "EUC-JP"
        soup = BeautifulSoup(html.content, "html.parser")
        
        all_chokyo=soup.find('div', attrs={'class': 'Unit_Race'}).find_all('li', attrs={'class': 'DB_Stable_Time'})

        
        df = pd.DataFrame()
        for i in range(len(all_chokyo)):
            try:
                chokyo = all_chokyo[i]


                a_tag = chokyo.find('a', href=True)
                href = a_tag['href']

                match = re.search(r'race_id=(\d+)', href)
                if match:
                    race_id = match.group(1)
                if not match:
                    match = re.search(r'race/(\d+)/', href)
                    if match:
                        race_id = match.group(1)


                # race_id = chokyo.find('a', attrs={'href': re.compile(r'race/(\d+)/')})['href'].split('/')[-2]

                cond_ls = chokyo.find_all('span', attrs={'class': 'Training_infoTxt'})
                cond_df = pd.DataFrame()
                for j in range(len(cond_ls)):
                    ls_temp =  [np.nan for _ in range(4)]
                    cond = cond_ls[j].text.split()
                    
                    cond[0] = cond[0].split('(')[0]
                    cond[0] = datetime.strptime(cond[0], '%Y/%m/%d').date()
                    for k in range(len(cond)):
                        ls_temp[k] = cond[k]
                    ls_temp.append(race_id)
                    cond_df = pd.concat([cond_df, pd.DataFrame(ls_temp).T])
                    
                df = pd.concat([df, pd.DataFrame(cond_df)])
            except IndexError:
                continue
            except AttributeError: #存在しないrace_idでAttributeErrorになるページもあるので追加
                continue
            #wifiの接続が切れた時などでも途中までのデータを返せるようにする
            except Exception as e:
                print(e)
                break
            #Jupyterで停止ボタンを押した時の対処
            except:
                break

        df.columns = ['date', 'place', 'condition', 'rider', 'race_id']


        laptime_df = pd.DataFrame()
        laptime_table = pd.read_html(html.text)
        for i in range(len(laptime_table)):
            laptime_df = pd.concat([laptime_df, laptime_table[i]])
        def to_float(val):
            try:
                return float(val)
            except ValueError:
                return np.nan
        laptime_df = laptime_df.replace(to_replace='\(.*?\)', value='', regex=True)
        laptime_df = laptime_df.replace('-', np.nan)
        laptime_df.iloc[:, :-2] = laptime_df.iloc[:, :-2].applymap(to_float)

        df = pd.concat([df, laptime_df], axis=1)

        df.index = [horse_id]*len(df)
        return df

    def chokyo_all_horse(self, horse_id_ls):
        df = pd.DataFrame()
        for horse_id in tqdm(horse_id_ls):
            time.sleep(0.5)
            try:
                df = pd.concat([df, self.chokyo_one_horse(horse_id)])
            except AttributeError:
                None_Value_df = pd.DataFrame([np.nan for _ in range(len(chokyo.columns))]).T
                None_Value_df.columns = chokyo.columns
                None_Value_df.index = [horse_id]
                df = pd.concat([df, None_Value_df])
            
        return df



In [ ]:
for i in range(0, 37000, 1000):    
    horse_result = pd.read_pickle('horse_result_raceID_added.pickle')
    horse_chokyo = pd.read_pickle(f'chokyo/chokyo_{i}.pickle')
    horse_result = horse_result.reset_index()
    horse_chokyo = horse_chokyo.reset_index()
    horse_result_chokyo = pd.merge(horse_result, horse_chokyo, on=['index', 'race_id'], how='right')
    horse_result_chokyo.index = horse_result_chokyo['index']
    horse_result_chokyo.index = horse_result_chokyo['index']
    horse_result_chokyo = horse_result_chokyo.drop('index', axis=1)
    horse_result_chokyo.to_pickle(f'horse_chokyo_Ver2/horse_result_chokyo_{i}.pickle')


In [2]:
horse_result_chokyo = pd.read_pickle('horse_chokyo_Ver2/horse_result_chokyo_0.pickle')
for i in tqdm(range(1000, 37000, 1000)):
    horse_result_chokyo_temp = pd.read_pickle(f'horse_chokyo_Ver2/horse_result_chokyo_{i}.pickle')
    horse_result_chokyo = pd.concat([horse_result_chokyo, horse_result_chokyo_temp])
horse_result_chokyo

  0%|          | 0/36 [00:00<?, ?it/s]

,日付,開催,天 気,R,レース名,映 像,頭 数,枠 番,馬 番,オ ッ ズ,...,place,condition,rider,0,1,2,3,4,5,6
index,,,,,,,,,,,,,,,,,,,,,
2015100713,2020/11/21,5阪神5,晴,11.0,アンドロメダS(L),NaN,15.0,1.0,1.0,68.8,...,南Ｗ,良,助手,NaN,73.7,57.6,43.0,13.6,馬也⑨,まずまずC
2015100713,2020/11/21,5阪神5,晴,11.0,アンドロメダS(L),NaN,15.0,1.0,1.0,68.8,...,南Ｗ,良,見習,NaN,65.4,51.4,38.8,12.8,馬也⑧,順調C
2015100713,2020/11/21,5阪神5,晴,11.0,アンドロメダS(L),NaN,15.0,1.0,1.0,68.8,...,南Ｗ,良,石川,NaN,NaN,53.2,39.6,12.9,馬也③,前走並みC
2015100713,2020/11/21,5阪神5,晴,11.0,アンドロメダS(L),NaN,15.0,1.0,1.0,68.8,...,南Ｗ,稍,助手,NaN,NaN,NaN,NaN,NaN,馬也⑦,順調C
2015100713,2020/11/21,5阪神5,晴,11.0,アンドロメダS(L),NaN,15.0,1.0,1.0,68.8,...,南Ｗ,良,助手,NaN,73.9,58.1,42.8,14.1,馬也⑥,まずまずC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020106815,2023/01/22,1小倉4,曇,5.0,3歳新馬,NaN,16.0,7.0,14.0,12.4,...,栗坂,良,助手,NaN,53.4,39.4,25.7,13.1,一杯①,反応平凡C
2020106815,2023/01/22,1小倉4,曇,5.0,3歳新馬,NaN,16.0,7.0,14.0,12.4,...,栗坂,良,NaN,NaN,54.5,39.4,25.1,12.4,馬也,順調C
2020106815,2023/01/22,1小倉4,曇,5.0,3歳新馬,NaN,16.0,7.0,14.0,12.4,...,栗Ｅ,良,助手,NaN,NaN,NaN,12.5,13.0,NaN,発馬鋭いB


In [21]:
horse_result_chokyo['chokyo_day_diff'] = pd.to_datetime(horse_result_chokyo['日付']) - pd.to_datetime(horse_result_chokyo['date'])

In [38]:
horse_result = pd.read_pickle('horse_result.pickle')
horse_result

,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,オッズ,...,着差,ﾀｲﾑ指数,通過,ペース,上り,馬体重,厩舎ｺﾒﾝﾄ,備考,勝ち馬(2着馬),賞金
2015100713,2020/11/21,5阪神5,晴,11.0,アンドロメダS(L),NaN,15.0,1.0,1,68.8,...,1.2,**,12-11-11-12,35.4-35.5,35.5,428(-6),NaN,NaN,アドマイヤビルゴ,NaN
2015100713,2020/10/24,4新潟5,雨,11.0,新潟牝馬S(OP),NaN,13.0,6.0,8,12.1,...,2.2,**,6-7-9-7,36.3-35.6,37.0,434(+6),NaN,NaN,ウラヌスチャーム,NaN
2015100713,2020/09/06,3新潟8,晴,11.0,新潟記念(G3),NaN,18.0,7.0,15,75.8,...,0.7,**,4-5,36.8-33.1,33.5,428(+4),NaN,NaN,ブラヴァス,NaN
2015100713,2020/08/02,1札幌4,晴,11.0,クイーンS(G3),NaN,14.0,6.0,10,16.8,...,0.3,**,10-10-10-11,34.8-36.0,35.1,424(+4),NaN,NaN,レッドアネモス,NaN
2015100713,2020/06/14,3阪神4,曇,11.0,マーメイドS(G3),NaN,16.0,4.0,8,10.1,...,1.2,**,10-10-8-8,36.3-36.5,36.8,420(-4),NaN,NaN,サマーセント,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020100674,2023/02/05,1中京12,晴,6.0,3歳新馬,NaN,15.0,5.0,8,20.3,...,0.7,**,9-10-10,37.4-34.5,34.2,400(0),NaN,NaN,タイキバルドル,62.0
2020106815,2023/05/14,1京都8,曇,4.0,3歳未勝利,NaN,10.0,5.0,5,27.1,...,0.6,**,10-9-9-9,35.8-37.7,37.1,464(+4),NaN,NaN,ニホンピロアリー,NaN
2020106815,2023/02/26,1阪神6,曇,5.0,3歳未勝利,NaN,17.0,7.0,14,75.0,...,0.7,**,13-13,36.5-35.0,34.8,460(-2),NaN,NaN,ルクスドヌーヴ,NaN
2020106815,2023/02/12,1阪神2,晴,6.0,3歳未勝利,NaN,12.0,1.0,1,23.5,...,1.0,**,2-1-1-1,35.3-36.6,37.6,462(-2),NaN,NaN,レシプロシティ,NaN


In [48]:
horse_result_chokyo = pd.read_pickle('horse_result_chokyo.pickle')
# Make sure 'date' is a datetime type.
horse_result_chokyo['date'] = pd.to_datetime(horse_result_chokyo['date'])

horse_result_chokyo = horse_result_chokyo.reset_index()
horse_result_chokyo = horse_result_chokyo.rename(columns={'index': 'horse_id'})

# # Sort values by 'race_id', 'horse_id' (the index), and 'date'.
# horse_result_chokyo = horse_result_chokyo.sort_values(['race_id', 'horse_id', 'date'])

# # Drop duplicates of 'race_id' and 'horse_id', keeping only the last entry.
# horse_result_chokyo = horse_result_chokyo.drop_duplicates(subset=['race_id', 'horse_id'], keep='last').set_index('horse_id')

horse_result_chokyo.groupby(['horse_id', 'race_id']).count()

日付  開催  天気  R  レース名  映像  頭数  枠番  馬番  オ ッ ズ  ...   
horse_id   race_id                                                   ...   
2006106754 200903020507   7   7   7  7     7   0   7   7   7      7  ...  \
           200903031009   3   3   3  3     3   0   3   3   3      3  ...   
           200904010704   4   4   4  4     4   0   4   4   4      4  ...   
           200905020306   3   3   3  3     3   0   3   3   3      3  ...   
           200905040208   7   7   7  7     7   0   7   7   7      7  ...   
...                      ..  ..  .. ..   ...  ..  ..  ..  ..    ...  ...   
2020110154 202205050806   8   8   8  8     8   0   8   8   8      8  ...   
           202309011205   5   5   5  5     5   0   5   5   5      5  ...   
2020110155 202205050705   9   9   9  9     9   0   9   9   9      9  ...   
           202209060611   3   3   3  3     3   0   3   3   3      3  ...   
           202305010611   7   7   7  7     7   0   7   7   7      7  ...   

                         condition  rider  6F  5F  4F  3F  1F  strength   
horse_id   race_id                                                        
2006106754 200903020507          7      6   0   6   7   7   6         7  \
           200903031009          3      3   0   3   3   3   3         3   
           200904010704          3      2   0   4   4   4   4         4   
           200905020306          3      2   0   3   3   3   3         3   
           200905040208          7      7   1   7   7   7   7         7   
...                            ...    ...  ..  ..  ..  ..  ..       ...   
2020110154 202205050806          8      5   2   8   8   8   8         8   
           202309011205          5      3   1   5   5   5   5         5   
2020110155 202205050705          9      9   4   8   9   9   9         8   
           202209060611          3      3   1   2   3   3   3         3   
           202305010611          7      7   4   7   7   7   7         7   

                         evaluation  chokyo_day_diff  
horse_id   race_id                                    
2006106754 200903020507           7                7  
           200903031009           3                3  
           200904010704           4                4  
           200905020306           3                3  
           200905040208           7                7  
...                             ...              ...  
2020110154 202205050806           8                8  
           202309011205           5                5  
2020110155 202205050705           9                9  
           202209060611           3                3  
           202305010611           7                7  

[378887 rows x 40 columns]

In [10]:
df = horse_result_chokyo.copy()
df['date'] = pd.to_datetime(df['date'])
df = df.dropna(subset=['date'])
df = df.loc[df.groupby([df.index, 'race_id'])['date'].idxmax()]
df

,日付,開催,天 気,R,レース名,映 像,頭 数,枠 番,馬 番,オ ッ ズ,...,place,condition,rider,0,1,2,3,4,5,6
index,,,,,,,,,,,,,,,,,,,,,
2006106754,2018/03/24,3中山1,晴,11.0,日経賞(G2),NaN,15.0,5.0,8.0,625.9,...,美坂,稍,助手,NaN,57.1,42.0,27.6,13.6,馬也②,平行線C
2006106754,2018/03/24,3中山1,晴,11.0,日経賞(G2),NaN,15.0,5.0,8.0,625.9,...,美坂,良,助手,NaN,57.3,42.1,27.7,13.7,馬也,変わらずC
2006106754,2018/03/24,3中山1,晴,11.0,日経賞(G2),NaN,15.0,5.0,8.0,625.9,...,美坂,良,助手,NaN,57.3,42.2,28.0,13.8,馬也,モタつくD
2006106754,2018/03/24,3中山1,晴,11.0,日経賞(G2),NaN,15.0,5.0,8.0,625.9,...,美坂,良,師,NaN,55.1,40.9,27.1,13.5,馬也,変わらずC
2006106754,2018/02/17,1東京7,晴,11.0,ダイヤモンドS(G3),NaN,14.0,7.0,11.0,403.8,...,南Ｗ,良,師,NaN,70.1,55.0,40.8,13.6,強め⑤,気配平凡C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020110155,2022/11/26,5東京7,曇,5.0,2歳新馬,NaN,17.0,8.0,17.0,4.4,...,南Ｗ,良,助手,88.1,70.6,54.8,39.2,11.5,馬也⑧,動き上々B
2020110155,2022/11/26,5東京7,曇,5.0,2歳新馬,NaN,17.0,8.0,17.0,4.4,...,美坂,良,助手,NaN,55.6,41.3,27.2,13.3,馬也,順調C
2020110155,2022/11/26,5東京7,曇,5.0,2歳新馬,NaN,17.0,8.0,17.0,4.4,...,南Ｗ,良,助手,83.1,67.5,52.4,38.4,12.4,馬也⑧,順調C


In [ ]:
race_id = '202305030205'
url_chokyo = 'https://race.netkeiba.com/race/oikiri.html?race_id=' + race_id + '&type=1&rf=shutuba_submenu'


user_id = 'ken.osechi@gmail.com'
password = 'keson0213'
login_info = {
    "login_id": user_id,
    "pswd": password,
}
session = requests.session()
url_login = "https://regist.netkeiba.com/account/?pid=login&action=auth"
ses = session.post(url_login, data=login_info)
cookie = ses.cookies

# url = 'https://db.sp.netkeiba.com/horse/training/' + horse_id

html = session.get(url_chokyo, cookies=cookie)
html.encoding = "EUC-JP"
soup = BeautifulSoup(html.content, "html.parser")


In [64]:
dd = pd.read_html(html.text)[0]
dd['調教タイム ラップ表示'] = dd['調教タイム ラップ表示'].replace('\(.*?\)', value=', ', regex=True)
floats = dd['調教タイム ラップ表示'].str.extractall('(\d+\.\d+)').unstack()
floats

0                        
match     0     1     2     3     4
0      86.9  68.9  53.2  38.4  11.5
1      52.9  38.2  24.6  12.2   NaN
2      97.9  67.6  53.0  38.5  11.8
3      54.2  39.4  25.6  13.1   NaN
4      86.4  69.2  53.9  39.4  11.7
..      ...   ...   ...   ...   ...
60     60.2  44.4  29.7  14.5   NaN
61     87.8  69.2  53.5  39.0  12.7
62     87.4  67.9  52.4  38.3  12.6
63     87.6  69.8  53.9  39.0  12.1
64     95.0  76.3  59.1  42.5  13.6

[65 rows x 5 columns]

In [74]:
horses = pd.read_pickle('horse_result.pickle')
horse_id_list = horses.index.unique()

chokyo = GetChokyo().chokyo_all_horse(horse_id_list[15713:15718])
chokyo[chokyo.index == '2015105814']

  0%|          | 0/5 [00:00<?, ?it/s]

,date,place,condition,rider,race_id,0,1,2,3,4,5,6


In [75]:
chokyo[chokyo.index == '2015105814']

,date,place,condition,rider,race_id,0,1,2,3,4,5,6
2015105814,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
# horse_id=horse_id_list[15715]
None_Value_df = pd.DataFrame([np.nan for _ in range(len(chokyo.columns))]).T
None_Value_df.columns = chokyo.columns
None_Value_df.index = [horse_id]
None_Value_df

,date,place,condition,rider,race_id,0,1,2,3,4,5,6
2015105814,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
user_id = 'ken.osechi@gmail.com'
password = 'keson0213'
login_info = {
    "login_id": user_id,
    "pswd": password,
}
session = requests.session()
url_login = "https://regist.netkeiba.com/account/?pid=login&action=auth"
ses = session.post(url_login, data=login_info)
cookie = ses.cookies


url = 'https://db.sp.netkeiba.com/horse/training/' + horse_id

html = session.get(url, cookies=cookie)
html.encoding = "EUC-JP"
soup = BeautifulSoup(html.content, "html.parser")
# all_chokyo=soup.find('div', attrs={'class': 'Unit_Race'}).find_all('li', attrs={'class': 'DB_Stable_Time'})

In [57]:
# all_chokyo=soup.find('div', attrs={'class': 'Unit_Race'}).find_all('li', attrs={'class': 'DB_Stable_Time'})
soup


<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<!-- <link rel="stylesheet" type="text/css" href="../../db_nk01.css"> -->
<!-- block=meta_tag (d) -->
<!--<base href=""> -->
<meta content="telephone=no" name="format-detection">
<meta content="all" name="robots"/>
<link href="https://cdn.netkeiba.com/img.spcommon/img/common/icon_home.png" rel="apple-touch-icon"/>
<!-- basic -->
<meta content="width=device-width,initial-scale=1.0,minimum-scale=1.0,maximum-scale=1.0,user-scalable=0" name="viewport"/>
<!-- block=meta_tag_db_horse_pid (cp) -->
<meta content="ja" http-equiv="content-language"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=0.1, user-scalable=yes" name="viewport"/>
<meta content="telephone=no" name="format-detection">
<meta content="グランドレビューの調教タイムです。競走成績、血統情報、産駒情報などをはじめ、50万頭以上の競走馬・騎手・調教師・馬主・生産者・レースの全データがご覧いただけます。" name="description">
<meta content="競馬情報,競走馬,騎手,レース,調教師,馬主,検索,データベース,JRA,netkeiba,ネット競馬" name="keywords">
<meta conte

In [2]:
chokyo = pd.read_pickle('chokyo.pickle')
horse_result = pd.read_pickle('horse_result.pickle')
lap = pd.read_pickle('lap.pickle')
race = pd.read_pickle('race.pickle')

In [3]:
race_lap = pd.merge(race, lap, left_index=True, right_index=True, how='left')
del race
del lap

In [4]:
horse_result_chokyo = pd.merge(horse_result, chokyo, left_index=True, right_index=True)

KeyboardInterrupt: 

In [17]:
chokyo = pd.read_pickle('chokyo.pickle')


,date,place,condition,rider,race_id,0,1,2,3,4,5,6
2015100713,2020-11-18,南Ｗ,良,助手,202009050511,NaN,73.7,57.6,43.0,13.6,馬也⑨,まずまずC
2015100713,2020-11-15,南Ｗ,良,見習,202009050511,NaN,65.4,51.4,38.8,12.8,馬也⑧,順調C
2015100713,2020-11-11,南Ｗ,良,石川,202009050511,NaN,NaN,53.2,39.6,12.9,馬也③,前走並みC
2015100713,2020-11-08,南Ｗ,稍,助手,202009050511,NaN,NaN,NaN,NaN,NaN,馬也⑦,順調C
2015100713,2020-11-05,南Ｗ,良,助手,202009050511,NaN,73.9,58.1,42.8,14.1,馬也⑥,まずまずC
...,...,...,...,...,...,...,...,...,...,...,...,...
2020106815,2023-01-12,栗坂,良,助手,202310010405,NaN,53.4,39.4,25.7,13.1,一杯①,反応平凡C
2020106815,2023-01-08,栗坂,良,NaN,202310010405,NaN,54.5,39.4,25.1,12.4,馬也,順調C
2020106815,2023-01-04,栗Ｅ,良,助手,202310010405,NaN,NaN,NaN,12.5,13.0,NaN,発馬鋭いB
2020106815,2022-12-31,ＣＷ,良,助手,202310010405,83.9,67.8,52.7,38.1,12.4,一杯⑥,順調C


In [9]:
import pandas as pd
race = pd.read_pickle('race.pickle')
horse = pd.read_pickle('horse_result.pickle')
chokyo = pd.read_pickle('chokyo.pickle')

In [10]:
chokyo.sort_values('race_id', inplace=True)

In [17]:
a1=chokyo['race_id'].unique()

In [25]:
a2=horse.drop_duplicates()

In [7]:
import pandas as pd
horse = pd.read_pickle('horse_result.pickle')
chokyo = pd.read_pickle('chokyo.pickle')

# i = 300
# a = pd.merge(horse.iloc[0:i, :], chokyo, left_index=True, right_index=True) 

In [15]:
sum(chokyo.index.unique() != horse.index.unique())

0

In [16]:
for i in range(0, len(horse), 50000):
    horse_chokyo = pd.merge(horse.iloc[i:i+50000, :], chokyo, left_index=True, right_index=True)
    horse_chokyo.to_pickle('horse_chokyo/horse_chokyo_' + str(i) + '.pickle')
    del horse_chokyo

In [3]:
import pandas as pd
import tqdm
horse_chokyo = pd.read_pickle('horse_chokyo/horse_chokyo_0.pickle')
for i in tqdm.tqdm(range(50000, 800000, 50000)):
    horse_chokyo_temp = pd.read_pickle('horse_chokyo/horse_chokyo_' + str(i) + '.pickle')
    horse_chokyo = pd.concat([horse_chokyo, horse_chokyo_temp])
horse_chokyo

100%|██████████| 15/15 [06:12<00:00, 24.83s/it]


,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,オッズ,...,condition,rider,race_id,0,1,2,3,4,5,6
2008102492,2018/02/03,2京都3,曇,9.0,稲荷特別(1000万下),NaN,10.0,3.0,3,222.6,...,良,助手,201808020309,84.3,68.5,53.8,39.8,12.9,馬也④,平行線C
2008102492,2018/02/03,2京都3,曇,9.0,稲荷特別(1000万下),NaN,10.0,3.0,3,222.6,...,稍,助手,201808020309,NaN,74.2,58.5,43.0,12.9,馬也⑧,順調C
2008102492,2018/02/03,2京都3,曇,9.0,稲荷特別(1000万下),NaN,10.0,3.0,3,222.6,...,稍,助手,201808010512,84.9,69.2,53.8,40.1,12.9,一杯⑦,変わらずC
2008102492,2018/02/03,2京都3,曇,9.0,稲荷特別(1000万下),NaN,10.0,3.0,3,222.6,...,良,助手,201709050810,NaN,69.0,54.5,40.6,13.3,一杯④,変わらずC
2008102492,2018/02/03,2京都3,曇,9.0,稲荷特別(1000万下),NaN,10.0,3.0,3,222.6,...,良,助手,201709050409,85.6,69.3,54.0,39.9,13.3,一杯⑥,伸び一息C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020110155,2022/11/26,5東京7,曇,5.0,2歳新馬,NaN,17.0,8.0,17,4.4,...,良,助手,202205050705,88.1,70.6,54.8,39.2,11.5,馬也⑧,動き上々B
2020110155,2022/11/26,5東京7,曇,5.0,2歳新馬,NaN,17.0,8.0,17,4.4,...,良,助手,202205050705,NaN,55.6,41.3,27.2,13.3,馬也,順調C
2020110155,2022/11/26,5東京7,曇,5.0,2歳新馬,NaN,17.0,8.0,17,4.4,...,良,助手,202205050705,83.1,67.5,52.4,38.4,12.4,馬也⑧,順調C
2020110155,2022/11/26,5東京7,曇,5.0,2歳新馬,NaN,17.0,8.0,17,4.4,...,良,助手,202205050705,NaN,54.1,39.6,26.3,13.5,馬也,順調C


In [4]:
horse_chokyo.to_pickle('horse_chokyo.pickle')

In [6]:
del horse_chokyo

In [28]:
horse_chokyo_temp['日付'] = pd.to_datetime(horse_chokyo_temp['日付']).dt.date
horse_chokyo_temp

,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,オッズ,...,condition,rider,race_id,0,1,2,3,4,5,6
2013106297,2022-08-24,笠松,晴,7.0,C11・C14組,NaN,9.0,8.0,8,63.6,...,良,助手,202206020404,NaN,53.9,41.3,28.5,15.0,一杯,伸び平凡D
2013106297,2022-08-24,笠松,晴,7.0,C11・C14組,NaN,9.0,8.0,8,63.6,...,良,NaN,202206020404,NaN,56.3,43.2,30.3,16.2,馬也,順調C
2013106297,2022-08-24,笠松,晴,7.0,C11・C14組,NaN,9.0,8.0,8,63.6,...,良,助手,201603010501,NaN,NaN,50.6,39.5,15.3,一杯④,伸び一息C
2013106297,2022-08-24,笠松,晴,7.0,C11・C14組,NaN,9.0,8.0,8,63.6,...,良,助手,201603010501,NaN,NaN,50.5,38.8,13.2,一杯⑥,伸び一息C
2013106297,2022-08-24,笠松,晴,7.0,C11・C14組,NaN,9.0,8.0,8,63.6,...,稍,助手,201603010501,NaN,NaN,52.3,40.1,13.9,馬也⑦,順調C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020110155,2022-11-26,5東京7,曇,5.0,2歳新馬,NaN,17.0,8.0,17,4.4,...,良,助手,202205050705,88.1,70.6,54.8,39.2,11.5,馬也⑧,動き上々B
2020110155,2022-11-26,5東京7,曇,5.0,2歳新馬,NaN,17.0,8.0,17,4.4,...,良,助手,202205050705,NaN,55.6,41.3,27.2,13.3,馬也,順調C
2020110155,2022-11-26,5東京7,曇,5.0,2歳新馬,NaN,17.0,8.0,17,4.4,...,良,助手,202205050705,83.1,67.5,52.4,38.4,12.4,馬也⑧,順調C
2020110155,2022-11-26,5東京7,曇,5.0,2歳新馬,NaN,17.0,8.0,17,4.4,...,良,助手,202205050705,NaN,54.1,39.6,26.3,13.5,馬也,順調C


In [ ]:
horse_chokyo_temp = horse_chokyo_temp[horse_chokyo_temp['date'] <= horse_chokyo_temp['日付']]

# # create a new column that is the '日付' column shifted down by one
# horse_chokyo_temp['previous_日付'] = horse_chokyo_temp['日付'].shift()

# # drop the first row because it will have a NaN 'previous_日付'
# horse_chokyo_temp = horse_chokyo_temp.dropna(subset=['previous_日付'])

# # drop the rows where 'date' is not in the range ['previous_日付', '日付']
# horse_chokyo_temp = horse_chokyo_temp[(horse_chokyo_temp['date'] >= horse_chokyo_temp['previous_日付']) & (horse_chokyo_temp['date'] <= horse_chokyo_temp['日付'])]

In [29]:
horse_chokyo_temp = horse_chokyo_temp[horse_chokyo_temp['date'] <= horse_chokyo_temp['日付']]
horse_chokyo_temp

,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,オッズ,...,condition,rider,race_id,0,1,2,3,4,5,6
2013106297,2022-08-24,笠松,晴,7.0,C11・C14組,NaN,9.0,8.0,8,63.6,...,良,助手,202206020404,NaN,53.9,41.3,28.5,15.0,一杯,伸び平凡D
2013106297,2022-08-24,笠松,晴,7.0,C11・C14組,NaN,9.0,8.0,8,63.6,...,良,NaN,202206020404,NaN,56.3,43.2,30.3,16.2,馬也,順調C
2013106297,2022-08-24,笠松,晴,7.0,C11・C14組,NaN,9.0,8.0,8,63.6,...,良,助手,201603010501,NaN,NaN,50.6,39.5,15.3,一杯④,伸び一息C
2013106297,2022-08-24,笠松,晴,7.0,C11・C14組,NaN,9.0,8.0,8,63.6,...,良,助手,201603010501,NaN,NaN,50.5,38.8,13.2,一杯⑥,伸び一息C
2013106297,2022-08-24,笠松,晴,7.0,C11・C14組,NaN,9.0,8.0,8,63.6,...,稍,助手,201603010501,NaN,NaN,52.3,40.1,13.9,馬也⑦,順調C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020110155,2022-11-26,5東京7,曇,5.0,2歳新馬,NaN,17.0,8.0,17,4.4,...,良,助手,202205050705,88.1,70.6,54.8,39.2,11.5,馬也⑧,動き上々B
2020110155,2022-11-26,5東京7,曇,5.0,2歳新馬,NaN,17.0,8.0,17,4.4,...,良,助手,202205050705,NaN,55.6,41.3,27.2,13.3,馬也,順調C
2020110155,2022-11-26,5東京7,曇,5.0,2歳新馬,NaN,17.0,8.0,17,4.4,...,良,助手,202205050705,83.1,67.5,52.4,38.4,12.4,馬也⑧,順調C
2020110155,2022-11-26,5東京7,曇,5.0,2歳新馬,NaN,17.0,8.0,17,4.4,...,良,助手,202205050705,NaN,54.1,39.6,26.3,13.5,馬也,順調C


In [48]:
def drop_invalid_dates(group):
    group['date_prev'] = 
    group['valid'] = group['date'] <= group['日付']
    return group[group['valid']]

horse_chokyo_temp_2 = horse_chokyo_temp.groupby(horse_chokyo_temp.index).apply(drop_invalid_dates)
horse_chokyo_temp_2

日付    開催 天気    R      レース名  映像    頭数   枠番  馬番   
2013106297 2013106297  2022-08-24    笠松  晴  7.0  C11・C14組 NaN   9.0  8.0   8  \
           2013106297  2022-08-24    笠松  晴  7.0  C11・C14組 NaN   9.0  8.0   8   
           2013106297  2022-08-24    笠松  晴  7.0  C11・C14組 NaN   9.0  8.0   8   
           2013106297  2022-08-24    笠松  晴  7.0  C11・C14組 NaN   9.0  8.0   8   
           2013106297  2022-08-24    笠松  晴  7.0  C11・C14組 NaN   9.0  8.0   8   
...                           ...   ... ..  ...       ...  ..   ...  ...  ..   
2020110155 2020110155  2022-11-26  5東京7  曇  5.0      2歳新馬 NaN  17.0  8.0  17   
           2020110155  2022-11-26  5東京7  曇  5.0      2歳新馬 NaN  17.0  8.0  17   
           2020110155  2022-11-26  5東京7  曇  5.0      2歳新馬 NaN  17.0  8.0  17   
           2020110155  2022-11-26  5東京7  曇  5.0      2歳新馬 NaN  17.0  8.0  17   
           2020110155  2022-11-26  5東京7  曇  5.0      2歳新馬 NaN  17.0  8.0  17   

                        オッズ  ...  rider       race_id     0     1     2     3   
2013106297 2013106297  63.6  ...     助手  202206020404   NaN  53.9  41.3  28.5  \
           2013106297  63.6  ...    NaN  202206020404   NaN  56.3  43.2  30.3   
           2013106297  63.6  ...     助手  201603010501   NaN   NaN  50.6  39.5   
           2013106297  63.6  ...     助手  201603010501   NaN   NaN  50.5  38.8   
           2013106297  63.6  ...     助手  201603010501   NaN   NaN  52.3  40.1   
...                     ...  ...    ...           ...   ...   ...   ...   ...   
2020110155 2020110155   4.4  ...     助手  202205050705  88.1  70.6  54.8  39.2   
           2020110155   4.4  ...     助手  202205050705   NaN  55.6  41.3  27.2   
           2020110155   4.4  ...     助手  202205050705  83.1  67.5  52.4  38.4   
           2020110155   4.4  ...     助手  202205050705   NaN  54.1  39.6  26.3   
           2020110155   4.4  ...     助手  202205050705   NaN   NaN  54.2  39.1   

                          4    5      6 valid  
2013106297 2013106297  15.0   一杯  伸び平凡D  True  
           2013106297  16.2   馬也    順調C  True  
           2013106297  15.3  一杯④  伸び一息C  True  
           2013106297  13.2  一杯⑥  伸び一息C  True  
           2013106297  13.9  馬也⑦    順調C  True  
...                     ...  ...    ...   ...  
2020110155 2020110155  11.5  馬也⑧  動き上々B  True  
           2020110155  13.3   馬也    順調C  True  
           2020110155  12.4  馬也⑧    順調C  True  
           2020110155  13.5   馬也    順調C  True  
           2020110155  12.1  馬也⑤    順調C  True  

[392791 rows x 41 columns]

In [1]:
import numpy as np
from tqdm import tqdm
import pandas as pd
horse_result = pd.read_pickle('horse_result.pickle')
len(horse_result)




780407

In [6]:
horse_chokyo_temp

,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,オッズ,...,condition,rider,race_id,0,1,2,3,4,5,6
2008102492,2018/02/03,2京都3,曇,9.0,稲荷特別(1000万下),NaN,10.0,3.0,3,222.6,...,良,助手,201808020309,84.3,68.5,53.8,39.8,12.9,馬也④,平行線C
2008102492,2018/02/03,2京都3,曇,9.0,稲荷特別(1000万下),NaN,10.0,3.0,3,222.6,...,稍,助手,201808020309,NaN,74.2,58.5,43.0,12.9,馬也⑧,順調C
2008102492,2018/02/03,2京都3,曇,9.0,稲荷特別(1000万下),NaN,10.0,3.0,3,222.6,...,稍,助手,201808010512,84.9,69.2,53.8,40.1,12.9,一杯⑦,変わらずC
2008102492,2018/02/03,2京都3,曇,9.0,稲荷特別(1000万下),NaN,10.0,3.0,3,222.6,...,良,助手,201709050810,NaN,69.0,54.5,40.6,13.3,一杯④,変わらずC
2008102492,2018/02/03,2京都3,曇,9.0,稲荷特別(1000万下),NaN,10.0,3.0,3,222.6,...,良,助手,201709050409,85.6,69.3,54.0,39.9,13.3,一杯⑥,伸び一息C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015110036,2017/06/24,1函館3,晴,6.0,2歳新馬,NaN,8.0,8.0,8,1.4,...,良,助手,201702010306,NaN,NaN,53.7,38.6,13.0,強め⑥,迫力十分B
2015110036,2017/06/24,1函館3,晴,6.0,2歳新馬,NaN,8.0,8.0,8,1.4,...,良,助手,201702010306,NaN,54.2,38.7,25.5,12.8,馬也,順調C
2015110036,2017/06/24,1函館3,晴,6.0,2歳新馬,NaN,8.0,8.0,8,1.4,...,良,助手,201702010306,NaN,NaN,54.3,39.0,12.1,強め⑧,順調C
2015110036,2017/06/24,1函館3,晴,6.0,2歳新馬,NaN,8.0,8.0,8,1.4,...,良,助手,201702010306,NaN,54.2,40.4,26.8,12.9,馬也,順調C


In [ ]:
horse_chokyo_temp = horse_chokyo_temp.copy()
for id in tqdm(horse_chokyo_temp.index.unique()[0:10]):
    hct2_temp = horse_chokyo_temp[horse_chokyo_temp.index == id]
    hiduke = sorted(hct2_temp['日付'].unique())
    for i, date in enumerate(hiduke):
        rows_to_update = hct2_temp[hct2_temp['日付'] == date].index
        if i == 0:
            horse_chokyo_temp.loc[rows_to_update, 'date_prev'] = np.nan
        else:
            horse_chokyo_temp.loc[rows_to_update, 'date_prev'] = hiduke[i-1]

# print(hiduke)
# hct2_temp
horse_chokyo_temp['date_prev'].unique()

In [24]:

def drop_invalid_date(path_name):
    horse_chokyo_temp = pd.read_pickle(path_name)
    flag=True
    for id in tqdm(horse_chokyo_temp.index.unique()):

        hct2_temp = horse_chokyo_temp[horse_chokyo_temp.index == id]
        hiduke = sorted(hct2_temp['日付'].unique())
        ls = []
        for row in hct2_temp.iterrows():
            # print(row[0], row[1]['日付'])
            
            if row[1]['日付'] == hiduke[0]:
                # print('a')
                ls.append(np.nan)
            else:
                # print('b', hiduke[hiduke.index(row[1]['日付'])-1])
                ls.append(hiduke[hiduke.index(row[1]['日付'])-1])

        if flag:
            date_prev = pd.DataFrame(ls, index=hct2_temp.index, columns=['date_prev'])
            flag=False

        else: date_prev = pd.concat([date_prev, pd.DataFrame(ls, index=hct2_temp.index, columns=['date_prev'])])


    horse_chokyo_temp = pd.concat([horse_chokyo_temp, date_prev], axis=1)
    horse_chokyo_temp['date_prev'] = pd.to_datetime(horse_chokyo_temp['date_prev']).dt.date
    horse_chokyo_temp = horse_chokyo_temp[horse_chokyo_temp['date_prev'] <= horse_chokyo_temp['date']]
    return horse_chokyo_temp

dropped_horse_chokyo = drop_invalid_date('horse_chokyo/horse_chokyo_0.pickle')
dropped_horse_chokyo


100%|██████████| 1441/1441 [09:06<00:00,  2.64it/s]


,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,オッズ,...,rider,race_id,0,1,2,3,4,5,6,date_prev
2008102492,2018/02/03,2京都3,曇,9.0,稲荷特別(1000万下),NaN,10.0,3.0,3,222.6,...,助手,201808020309,84.3,68.5,53.8,39.8,12.9,馬也④,平行線C,2018-01-14
2008102492,2018/02/03,2京都3,曇,9.0,稲荷特別(1000万下),NaN,10.0,3.0,3,222.6,...,助手,201808020309,NaN,74.2,58.5,43.0,12.9,馬也⑧,順調C,2018-01-14
2008102492,2018/01/14,1京都5,曇,12.0,4歳以上1000万下,NaN,13.0,6.0,8,138.4,...,助手,201808020309,84.3,68.5,53.8,39.8,12.9,馬也④,平行線C,2017-12-24
2008102492,2018/01/14,1京都5,曇,12.0,4歳以上1000万下,NaN,13.0,6.0,8,138.4,...,助手,201808020309,NaN,74.2,58.5,43.0,12.9,馬也⑧,順調C,2017-12-24
2008102492,2018/01/14,1京都5,曇,12.0,4歳以上1000万下,NaN,13.0,6.0,8,138.4,...,助手,201808010512,84.9,69.2,53.8,40.1,12.9,一杯⑦,変わらずC,2017-12-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015110036,2017/07/23,2函館6,晴,11.0,函館2歳S(G3),NaN,15.0,7.0,14,NaN,...,助手,201701020610,NaN,68.8,53.4,38.6,12.4,一杯⑤,動き軽快B,2017-06-24
2015110036,2017/07/23,2函館6,晴,11.0,函館2歳S(G3),NaN,15.0,7.0,14,NaN,...,助手,201701020610,NaN,NaN,56.0,40.9,12.9,仕掛⑦,順調C,2017-06-24
2015110036,2017/07/23,2函館6,晴,11.0,函館2歳S(G3),NaN,15.0,7.0,14,NaN,...,吉田隼,201702020611,NaN,67.0,50.4,37.1,12.2,強め⑧,迫力十分B,2017-06-24
2015110036,2017/07/23,2函館6,晴,11.0,函館2歳S(G3),NaN,15.0,7.0,14,NaN,...,吉田隼,201702020611,NaN,64.0,49.3,37.0,12.5,強め⑧,順調C,2017-06-24


In [26]:
# dropped_horse_chokyo.to_pickle('dropped_horse_chokyo/dropped_horse_chokyo.pickle')
# dropped_horse_chokyo.to_csv('dropped_horse_chokyo/dropped_horse_chokyo.csv')

In [22]:
flag=True
for id in tqdm(horse_chokyo_temp.index.unique()):

    hct2_temp = horse_chokyo_temp[horse_chokyo_temp.index == id]
    hiduke = sorted(hct2_temp['日付'].unique())
    ls = []
    for row in hct2_temp.iterrows():
        # print(row[0], row[1]['日付'])
        
        if row[1]['日付'] == hiduke[0]:
            # print('a')
            ls.append(np.nan)
        else:
            # print('b', hiduke[hiduke.index(row[1]['日付'])-1])
            ls.append(hiduke[hiduke.index(row[1]['日付'])-1])
   
    if flag:
        date_prev = pd.DataFrame(ls, index=hct2_temp.index, columns=['date_prev'])
        flag=False

    else: date_prev = pd.concat([date_prev, pd.DataFrame(ls, index=hct2_temp.index, columns=['date_prev'])])


horse_chokyo_temp = horse_chokyo_temp.copy()
horse_chokyo_temp = pd.concat([horse_chokyo_temp, date_prev], axis=1)
horse_chokyo_temp['date_prev'] = pd.to_datetime(horse_chokyo_temp['date_prev']).dt.date
horse_chokyo_temp = horse_chokyo_temp[horse_chokyo_temp['date_prev'] <= horse_chokyo_temp['date']]
horse_chokyo_temp

,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,オッズ,...,rider,race_id,0,1,2,3,4,5,6,date_prev
2008102492,2018/02/03,2京都3,曇,9.0,稲荷特別(1000万下),NaN,10.0,3.0,3,222.6,...,助手,201808020309,84.3,68.5,53.8,39.8,12.9,馬也④,平行線C,2018-01-14
2008102492,2018/02/03,2京都3,曇,9.0,稲荷特別(1000万下),NaN,10.0,3.0,3,222.6,...,助手,201808020309,NaN,74.2,58.5,43.0,12.9,馬也⑧,順調C,2018-01-14
2008102492,2018/01/14,1京都5,曇,12.0,4歳以上1000万下,NaN,13.0,6.0,8,138.4,...,助手,201808020309,84.3,68.5,53.8,39.8,12.9,馬也④,平行線C,2017-12-24
2008102492,2018/01/14,1京都5,曇,12.0,4歳以上1000万下,NaN,13.0,6.0,8,138.4,...,助手,201808020309,NaN,74.2,58.5,43.0,12.9,馬也⑧,順調C,2017-12-24
2008102492,2018/01/14,1京都5,曇,12.0,4歳以上1000万下,NaN,13.0,6.0,8,138.4,...,助手,201808010512,84.9,69.2,53.8,40.1,12.9,一杯⑦,変わらずC,2017-12-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015110036,2017/07/23,2函館6,晴,11.0,函館2歳S(G3),NaN,15.0,7.0,14,NaN,...,助手,201701020610,NaN,68.8,53.4,38.6,12.4,一杯⑤,動き軽快B,2017-06-24
2015110036,2017/07/23,2函館6,晴,11.0,函館2歳S(G3),NaN,15.0,7.0,14,NaN,...,助手,201701020610,NaN,NaN,56.0,40.9,12.9,仕掛⑦,順調C,2017-06-24
2015110036,2017/07/23,2函館6,晴,11.0,函館2歳S(G3),NaN,15.0,7.0,14,NaN,...,吉田隼,201702020611,NaN,67.0,50.4,37.1,12.2,強め⑧,迫力十分B,2017-06-24
2015110036,2017/07/23,2函館6,晴,11.0,函館2歳S(G3),NaN,15.0,7.0,14,NaN,...,吉田隼,201702020611,NaN,64.0,49.3,37.0,12.5,強め⑧,順調C,2017-06-24


In [23]:
horse_chokyo_temp

,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,オッズ,...,condition,rider,race_id,0,1,2,3,4,5,6
2008102492,2018/02/03,2京都3,曇,9.0,稲荷特別(1000万下),NaN,10.0,3.0,3,222.6,...,良,助手,201808020309,84.3,68.5,53.8,39.8,12.9,馬也④,平行線C
2008102492,2018/02/03,2京都3,曇,9.0,稲荷特別(1000万下),NaN,10.0,3.0,3,222.6,...,稍,助手,201808020309,NaN,74.2,58.5,43.0,12.9,馬也⑧,順調C
2008102492,2018/02/03,2京都3,曇,9.0,稲荷特別(1000万下),NaN,10.0,3.0,3,222.6,...,稍,助手,201808010512,84.9,69.2,53.8,40.1,12.9,一杯⑦,変わらずC
2008102492,2018/02/03,2京都3,曇,9.0,稲荷特別(1000万下),NaN,10.0,3.0,3,222.6,...,良,助手,201709050810,NaN,69.0,54.5,40.6,13.3,一杯④,変わらずC
2008102492,2018/02/03,2京都3,曇,9.0,稲荷特別(1000万下),NaN,10.0,3.0,3,222.6,...,良,助手,201709050409,85.6,69.3,54.0,39.9,13.3,一杯⑥,伸び一息C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015110036,2017/06/24,1函館3,晴,6.0,2歳新馬,NaN,8.0,8.0,8,1.4,...,良,助手,201702010306,NaN,NaN,53.7,38.6,13.0,強め⑥,迫力十分B
2015110036,2017/06/24,1函館3,晴,6.0,2歳新馬,NaN,8.0,8.0,8,1.4,...,良,助手,201702010306,NaN,54.2,38.7,25.5,12.8,馬也,順調C
2015110036,2017/06/24,1函館3,晴,6.0,2歳新馬,NaN,8.0,8.0,8,1.4,...,良,助手,201702010306,NaN,NaN,54.3,39.0,12.1,強め⑧,順調C
2015110036,2017/06/24,1函館3,晴,6.0,2歳新馬,NaN,8.0,8.0,8,1.4,...,良,助手,201702010306,NaN,54.2,40.4,26.8,12.9,馬也,順調C


In [ ]:
import pandas as pd
d = pd.read_pickle('race.pickle')
p = pd.read_pickle('peds.pickle').reset_index()
d = pd.merge(d, p, left_on='horse_id', right_on='index', how='left')


In [ ]:
for 

In [14]:
columns_to_check = ['peds_' + str(i) for i in range(62)]

# Create a boolean mask to identify rows with the desired value
mask = d[columns_to_check].apply(lambda x: 'モーリス 2011 鹿毛 [血統][産駒] Roberto系' in x.values, axis=1)

# Extract rows that satisfy the condition
filtered_rows = d[mask]
# filtered_rows[filtered_rows['性齢']=='牡2'].to_csv('filtered_rows.csv')

In [15]:
filtered_rows

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,タイム指数,...,peds_52,peds_53,peds_54,peds_55,peds_56,peds_57,peds_58,peds_59,peds_60,peds_61
134426,1,6,6,テーオーメアリー,牝2,54.0,ルメール,1:09.9,NaN,81.0,...,Ack Ack,Loyal Ruler,Buckpasser,Stepping High,In Reality,Rather Special,Northern Dancer,Twill,Rising Market,Always Lucky
134771,2,7,7,カランドゥーラ,牡2,54.0,ルメール,1:50.1,1/2,89.0,...,Busted,Highclere,Kalamoun,State Pension,Hornbeam,Priddy Fair,Northern Dancer,Lady Victoria,ガーサント,パロクサイド
134958,8,4,4,トウカイクラルテ,牡2,52.0,亀田温心,1:55.0,クビ,7.0,...,Blakeney,The Dancer,Hail to Reason,Cosmah,Understanding,Mountain Flower,Northern Dancer,Nice Princess,ファバージ,トウカイクイン
135126,2,1,1,エスコバル,牡2,54.0,ルメール,2:03.4,3/4,82.0,...,Blakeney,The Dancer,Halo,Wishing Well,Le Fabuleux,Marston's Mill,Vice Regent,Miss Attractive,マルゼンスキー,ホクトヒシヨウ
135159,1,8,13,カイザーノヴァ,牡2,54.0,坂井瑠星,1:29.6,NaN,85.0,...,セントクレスピン,ミスアシヤガワ,Mr. Prospector,Secrettame,The Minstrel,Mofida,Northern Dancer,Fairy Bridge,Ela-Mana-Mou,Welsh Flame
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272289,6,2,4,ルパルテール,牝4,53.0,田口貫太,1:25.9,クビ,80.0,...,Northern Dancer,Far,Lorenzaccio,Helen Nichols,Alleged,Rose Red,フォルティノ,Chambord,Buckpasser,Aspidistra
272300,1,3,6,アドヴァイス,牝4,56.0,幸英明,1:08.7,NaN,89.0,...,Crimson Satan,Consentida,Northern Dancer,Sex Appeal,Mill Reef,Irish Lass,ノーザンテースト,ロイヤルハイブ,インターメゾ,ゴールドアイアン
272328,2,5,6,アンテロース,牡3,56.0,吉田隼人,2:00.0,クビ,89.0,...,Busted,Highclere,テスコボーイ,アンジェリカ,ノーザンテースト,クリアアンバー,Icecapade,Bushel-n-Peck,Mo Exception,Double Wiggle
272359,5,2,2,ルペルカーリア,牡5,57.0,岩田望来,1:46.5,1,91.0,...,セントクレスピン,ミスアシヤガワ,Nearctic,Natalma,Bold Reason,Special,Sir Gaylord,Little Hut,Le Fabuleux,Pia
